## 偵測瑜珈墊在腳的位置

- 詳細介紹可參考[論文](https://docs.google.com/document/d/1sWPlbKvfi4x-Idih0DI4toHYrIhyDv6y/edit?usp=sharing&ouid=114571548892193624282&rtpof=true&sd=true) : 3.1. 瑜珈墊腳座標校正 

### 流程

![image](https://hackmd.io/_uploads/rJhu2ozIC.png)

1. 抓到瑜珈墊在鏡頭的座標點
    - 預設: [[0.9, 0.97],[0.1, 0.97],[0.15, 0.76],[0.85, 0.76]]
    - 實際抓: YogaMatRangeGetter.py
    - 目前專案是使用預設抓法， YogaMatRangeGetter.py 不一定穩定
2. 使用瑜伽墊的四個座標點，來建立透視變換的轉換矩陣
3. 將雙腳投影到瑜珈墊座標



### 1. python class 方置區

此區放置之後回移植到 android studio 的 python 檔案
在完成研究後，記得把程式碼複製上去

- ScoreCalculator.py

In [1]:
from FeetData import FeetData
import AngleNodeDef
import cv2
import numpy as np

from YogaMatRangeGetter import YogaMatRangeGetter


import cv2
import numpy as np


# 負責將人體骨架的腳座標點，轉換成瑜珈墊上面的點
# 轉至技術介紹: https://blog.csdn.net/guduruyu/article/details/72518340
class YogaMatProcessor:
    def __init__(self, use_default_matrix=True):
        # 鏡頭的座標位置
        self.camera_flat_points = np.array([[1, 1], [0, 1], [0, 0], [1, 0]], dtype=np.float32)
        # 預設瑜珈墊在鏡頭的座標
        default_mat_point = np.array([[0.9, 0.97],
                                      [0.1, 0.97],
                                      [0.15, 0.76],
                                      [0.85, 0.76]], dtype=np.float32)

        # 使用預設的轉至矩陣
        self.use_default_matrix = use_default_matrix
        # 轉換矩陣
        self.transform_matrix = cv2.getPerspectiveTransform(default_mat_point,
                                                            self.camera_flat_points)
        self.feet_data = FeetData()

    # 產生腳的資料
    def generate_feet_data(self, r_point2d, r_point3d):
        # 將骨架資料根據 python 格式進行轉換
        point2d, point3d = self.__handle_skeleton_point(r_point2d, r_point3d)

        # 檢查骨架是否為空
        if self.__contain_point(point2d, point3d):
            # 取得腳的 2d 座標
            feet_points = self.__get_feet_points(point2d)
            # 將腳的座標進行轉換
            transform_points = self.transform_point(feet_points)

            # 創建腳的資料
            left_feet, right_feet = transform_points[0], transform_points[1]
            self.feet_data.set_point(left_feet, right_feet)
        else:
            self.feet_data.set_point(None, None)

        return self.feet_data.to_dict()

    # 產稱瑜珈墊的轉至矩陣
    def generate_transform_matrix_with_image(self, image):
        range_getter = YogaMatRangeGetter()
        max_contour, approx, mat_points, contours, mask = range_getter.generate_mat_range(image, True)

        self.generate_transform_matrix(mat_points)

    # 產稱瑜珈墊的轉至矩陣
    def generate_transform_matrix(self, unit_points):
        is_4_point = len(unit_points[0]) == 4

        transform_matrix = cv2.getPerspectiveTransform(unit_points, self.camera_flat_points) if is_4_point else []

        if is_4_point:
            self.transform_matrix = transform_matrix

    # 將鏡頭中的座標點，轉換到瑜珈墊上面
    def transform_point(self, input_points):
        transformed_points = [
            cv2.perspectiveTransform(np.array([np.float32([p])]), self.transform_matrix)[0][0] for p
            in input_points]

        return transformed_points

    # 將骨架資料根據 python 格式進行轉換
    def __handle_skeleton_point(self, r_point2d, r_point3d):
        point3d = [[r_point3d.get(i).get(j) for j in range(3)] for i in range(r_point3d.size())]
        point2d = [[r_point2d.get(i).get(j) for j in range(2)] for i in range(r_point2d.size())]

        return point2d, point3d

    # 檢查骨架是否為空
    def __contain_point(self, point2d, point3d):
        return not isinstance(point2d, int) and not isinstance(point3d, int)

    # 從 MediaPipe 的點中，取得在鏡頭中腳的點
    def __get_feet_points(self, point2d):
        return [[point2d[AngleNodeDef.LEFT_HEEL][0], point2d[AngleNodeDef.LEFT_HEEL][1]],
                [point2d[AngleNodeDef.RIGHT_HEEL][0], point2d[AngleNodeDef.RIGHT_HEEL][1]]]

    # 取得腳的資料
    def get_left_foot_x(self):
        # 取得 left_foot 的 x 座標
        return self.feet_data.left_foot[0] if self.feet_data.left_foot is not None else - 999999

    def get_left_foot_y(self):
        # 取得 left_foot 的 y 座標
        return self.feet_data.left_foot[1] if self.feet_data.left_foot is not None else - 999999

    def get_right_foot_x(self):
        # 取得 right_foot 的 x 座標
        return self.feet_data.right_foot[0] if self.feet_data.right_foot is not None else - 999999

    def get_right_foot_y(self):
        # 取得 right_foot 的 y 座標
        return self.feet_data.right_foot[1] if self.feet_data.right_foot is not None else - 999999

In [10]:
def get_feet_points(point2d):
    if not isinstance(point2d, int):
        left_point = point2d[AngleNodeDef.LEFT_HEEL]
        right_point = point2d[AngleNodeDef.RIGHT_HEEL]
        return [[left_point.x, left_point.y], [right_point.x, right_point.y]]
    else:
        return []

#  將輸入的座標乘以 height 和 width
def scale_coordinates(input_point, width, height):
    return [[point[0] * width, point[1] * height] for point in input_point]

def inverse_transform_coordinates(coordinates, width, height):
    """
    將輸入的座標除以 height 和 width，返回反向轉換後的座標。

    Args:
        coordinates (numpy.ndarray): 二維座標陣列，每個元素都是一個座標點的陣列，如[[x1, y1], [x2, y2], ...]
        height (float): 高度
        width (float): 寬度

    Returns:
        numpy.ndarray: 反向轉換後的座標陣列，格式與輸入相同
    """
    """
    將輸入的座標除以 height 和 width，返回反向轉換後的座標，並四捨五入到小數第二位。

    Args:
        coordinates (numpy.ndarray): 二維座標陣列，每個元素都是一個座標點的列表，如[[x1, y1], [x2, y2], ...]
        height (float): 高度
        width (float): 寬度

    Returns:
        numpy.ndarray: 反向轉換後的座標陣列，格式與輸入相同
    """
    transformed_coordinates = coordinates / np.array([width, height], dtype=np.float32)
    return np.round(transformed_coordinates, decimals=2)

In [9]:

from develop_tool import  get_Mediapipe_point, resize_image

import cv2
import numpy as np

# 劃出在攝影機中，瑜珈墊框起來的狀況
def draw_camera_result(image, contours, max_contour, approx, raw_points):
    image_copy = image.copy()

    for contour in contours:
        cv2.drawContours(image_copy, [contour], 0, (255, 0, 0), 2)
        
    cv2.drawContours(image_copy, [max_contour], 0, (0, 255, 0), 2)
    cv2.drawContours(image_copy, [approx], -1, (0, 0, 255), 2)

    # Iterate through the points with corresponding colors
    for i, raw_point in enumerate(raw_points):
        # Draw on the original image
        cv2.circle(image_copy, tuple(map(int, raw_point)), 10, colors[i], thickness=cv2.FILLED)
    
    return image_copy


colors = [(0,0,255), (255, 0, 0)]

range_getter = YogaMatRangeGetter()
mat_processor = YogaMatProcessor()

lower_green = np.array([25, 20, 20])
upper_green = np.array([110, 255, 255])

range_getter.set_mask(lower_green, upper_green)


def display(image, file_name):
    height = len(image)
    width = len(image[0])
    # get point
    point2d, point3d = get_Mediapipe_point(image, False)
    input_point = get_feet_points(point2d)
    scale_point = scale_coordinates(input_point, width, height)
    print("input", input_point, "scale", scale_point)
    
    # create transform_matrix
    max_contour, approx, unit_point, contours , mask = range_getter.generate_mat_range(image)
    unit_points = inverse_transform_coordinates(unit_point, width, height)
    
    mat_processor.generate_transform_matrix(unit_points)
    transform_point = mat_processor.transform_point(input_point)
    
    user_frame =   draw_camera_result(image, [max_contour], max_contour, approx, scale_point)
    scale_transform_point = scale_coordinates(transform_point, width, height)
    # print("input_point", input_point)
    # print("scale_point", scale_point)
    # print("transform_point", transform_point)
    # print("scale", scale_transform_point)
    yoga_mat_image = draw_yoga_mat_result(height, width, scale_transform_point)
    # cv2.drawContours(image, contours, -1, (0,255,0), 3)
    
    full_image = cv2.hconcat([user_frame, yoga_mat_image])
    
    cv2.imshow('image' + file_name , full_image)
    # cv2.imshow('counter', mask)
    
    show_transform_mat(image, scale_point)

def show_transform_mat(image, input_point):
    # Create mat image
    h = 300
    w = 800
    
    max_contour, approx, unit_point, contours , mask = range_getter.generate_mat_range(image)
    
    mat_processor.camera_flat_points = np.array([[w, h], [0, h], [0, 0], [w, 0]], dtype=np.float32)
    mat_processor.generate_transform_matrix(unit_point)
    transform_image = cv2.warpPerspective(image, mat_processor.transform_matrix, (w, h))
    transform_point = mat_processor.transform_point(input_point)
    print("unit_point", unit_point)
    print("transform", transform_point)
    
    for i, raw_point in enumerate(transform_point):
        # Draw on the original image
        cv2.circle(transform_image, tuple(map(int, raw_point)), 10, colors[i], thickness=cv2.FILLED)
        
    cv2.imshow("transform_image", transform_image)


def show_video():
    video_path = "data/video/you.mp4"
    cap = cv2.VideoCapture(video_path)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Video end")
            break
            
        display(frame, "video")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()


def show_image():
    # "mat_test2.jpg"  "ewang.png" "cobra.png"
    image_names = ["ewang.png"]
    image_path = "data/image/"
    
    for name in image_names:
        image = cv2.imread(image_path + name, cv2.IMREAD_UNCHANGED)
        display(image, name)

    # 按下任意鍵則關閉所有視窗
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == "__main__":
    show_image()
    # show_video()


1280 720


TypeError: cannot unpack non-iterable NoneType object